## Azure Databricks / Azure Machine Learning Sample - Model Training & Deployment Staging

Sample notebook showcasing how to train an ML model in Azure Databricks and log/register in a target Azure Machine Learning workspace after performing a champion vs. challenger model evaluation. Further, this notebook contains sample code for deploying your newly trained model to a Managed Online Endpoint. The logic here will identify whether there is currently a model deployed, and if so will update the unoccupied blue/green spot, run a test, and automatically mirror 20% of traffic to the new endpoint.

This routine builds a new regression model around the Diabetes Dataset which is available as part of the Scikit-Learn library, and can be adapted to support training building upon existing datasets.

#### Import required packages

In [ ]:
import numpy as np
import os
from pyspark.sql import SparkSession  
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType  
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from sklearn import neighbors
from mlflow.deployments import get_deploy_client
import json
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Environment, ManagedOnlineDeployment, CodeConfiguration, TargetUtilizationScaleSettings
from azure.identity import DefaultAzureCredential
import time

#### Parse arguments

In [ ]:
subscription_id = dbutils.widgets.get('subscription_id')
resource_group = dbutils.widgets.get('resource_group')
workspace = dbutils.widgets.get('workspace')

model_name = dbutils.widgets.get('model_name')
endpoint_name = dbutils.widgets.get('endpoint_name')
endpoint_description = dbutils.widgets.get('endpoint_description')

#### Model training helper functions

Collection of functions to support model training, testing, and registration. These functions can be extended to support different types of ML models.

In [ ]:
def add_data_to_delta_table(path='dbfs:/tmp/gold-data/diabetes-features'):
    """
    This function loads the diabetes dataset, converts it to a Pandas DataFrame, and writes it to a Delta table.

    Args:  
        path (str, optional): The path where the Delta table will be saved. Defaults to 'dbfs:/tmp/gold-data/diabetes-features'.  

    Returns:  
        None  
    """  
    from delta.tables import DeltaTable
    from sklearn.datasets import load_diabetes
    import pandas as pd
    from pyspark.sql import SparkSession
    diabetes = load_diabetes()
    
    # Load the diabetes dataset  
    diabetes = load_diabetes()
    
    # Convert the dataset to a Pandas dataframe  
    df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)  
    df['target'] = diabetes.target  
     
    # Create a Delta table from the Pandas dataframe  
    spark = SparkSession.builder.getOrCreate()
    spark_df = spark.createDataFrame(df)
    spark_df.write.format('delta').mode('overwrite').save(path)  
    print(f"Data added to delta table at {path}")
    
    
def get_data_from_delta_table(path='dbfs:/tmp/gold-data/diabetes-features'):
    """
    This function reads data from a Delta table and returns it as a Pandas DataFrame.

    Args:  
        path (str, optional): The path to the Delta table to be read. Defaults to 'dbfs:/tmp/gold-data/diabetes-features'.  

    Returns:  
        pandas_df (pd.DataFrame): The data from the Delta table as a Pandas DataFrame.  
    """  
    import pandas as pd
    from pyspark.sql import SparkSession
    delta_data = spark.read.format("delta").load(path)  
    pandas_df = delta_data.toPandas()
    print(f"Loading data from delta table at {path}")
    return pandas_df

def get_aml_client(subscription_id, resource_group, workspace):
    """
    This function establishes a connection to an Azure Machine Learning (AML) workspace using a service principal.
    It retrieves the tenant ID, client ID, and client secret from a Databricks secret scope and returns an AML client object.

    Args:  
        subscription_id (str): The Azure subscription ID.  
        resource_group (str): The Azure resource group name.  
        workspace (str): The Azure Machine Learning workspace name.  

    Returns:  
        ml_client (azure.ml.core.client.MLClient): An Azure ML client object with an established connection to the specified AML workspace.  
    """  
    from azure.identity import ClientSecretCredential, DefaultAzureCredential
    import os

    tenant_id = dbutils.secrets.get(scope="amlsecretscope",key="tenantid")
    client_id = dbutils.secrets.get(scope="amlsecretscope",key="clientid")
    client_secret = dbutils.secrets.get(scope="amlsecretscope",key="clientsecret")
    
    os.environ["AZURE_TENANT_ID"] = tenant_id
    os.environ["AZURE_CLIENT_ID"] = client_id
    os.environ["AZURE_CLIENT_SECRET"] = client_secret

    credential = ClientSecretCredential(tenant_id, client_id, client_secret)

    ml_client = MLClient(
        credential, subscription_id, resource_group, workspace
    )
    print("Establishing connection to Azure ML workspace")
    return ml_client
    

def get_aml_mlflow_tracking_uri(ml_client):
    """
    This function retrieves the MLflow tracking URI for an Azure Machine Learning workspace.

    Args:  
        ml_client (object): The ml_client which references the target Azure Machine Learning workspace.  

    Returns:  
        tracking_uri (str): The MLflow tracking URI associated with the Azure Machine Learning workspace.  
    """  
    
    ws = ml_client.workspaces.get(workspace)
    return ws.mlflow_tracking_uri

def split_train_test_data(pandas_df, training_percent=0.8):
    """
    This function splits a Pandas DataFrame into training and testing datasets using the specified training percentage.

    Args:  
        pandas_df (pd.DataFrame): The input Pandas DataFrame to be split.  
        training_percent (float, optional): The percentage of data to be used for training. Defaults to 0.8.  

    Returns:  
        train_df (pd.DataFrame): The training dataset as a Pandas DataFrame.  
        test_df (pd.DataFrame): The testing dataset as a Pandas DataFrame.  
    """  
    from sklearn.model_selection import train_test_split
    train_df, test_df = train_test_split(pandas_df, test_size = 1.0 - training_percent)
    print('Splitting data into train/test subsets')
    return train_df, test_df
    
def train_regression_model(experiment_name, run_name, training_data, testing_data, target, tracking_uri):
    """
    This function trains a regression model using GradientBoostingRegressor and logs the run using MLflow.

    Args:  
        experiment_name (str): The name of the MLflow experiment.  
        run_name (str): The name of the MLflow run.  
        training_data (pd.DataFrame): The training dataset as a Pandas DataFrame.  
        testing_data (pd.DataFrame): The testing dataset as a Pandas DataFrame.  
        target (str): The name of the target variable in the datasets.  
        tracking_uri (str): The MLflow tracking URI.  

    Returns:  
        run_id (str): The MLflow run ID for the trained model.  
    """  
    from sklearn.model_selection import train_test_split
    from sklearn import preprocessing
    import mlflow
    import mlflow.sklearn
    from sklearn.pipeline import Pipeline
    from sklearn.ensemble import GradientBoostingRegressor
    
    mlflow.set_tracking_uri(tracking_uri)

    mlflow.sklearn.autolog()
    mlflow.set_experiment(experiment_name)
    
    X_train = training_data.drop(columns=[target])
    X_test = testing_data.drop(columns=[target])
    y_train = training_data[target]
    y_test = testing_data[target]

    with mlflow.start_run(run_name=run_name) as run:
        run_id = run.info.run_id

        scaler = preprocessing.MinMaxScaler()
        model = GradientBoostingRegressor()
        pipeline = Pipeline([('transformer', scaler), ('estimator', model)])
        pipeline.fit(X_train, y_train)
        
        print(f"Training and logging regression model. {run_id}")
    
    return run_id

def evaluate_challenger_model(run_id, model_name, test_df, target):
    """
    This function evaluates a challenger model against the current champion model using root mean squared error (RMSE).
    The challenger model is considered better if its RMSE is lower than or equal to the champion model's RMSE. The function
    also prints the RMSE values of both models for comparison.

    Args:  
        run_id (str): The MLflow run ID of the challenger model.  
        model_name (str): The name of the champion model registered in the MLflow model registry.  
        test_df (pd.DataFrame): The testing dataset as a Pandas DataFrame.  
        target (str): The name of the target variable in the test dataset.  

    Returns:  
        bool: True if the challenger model's RMSE is lower than or equal to the champion model's RMSE, False otherwise.  
    """  
    from sklearn.metrics import mean_squared_error
    try:
        champion_model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")
        challenger_model = mlflow.sklearn.load_model(f"runs:/{run_id}/model")
        
        X_test = test_df.drop(columns=[target])
        y_test = test_df[[target]]
        
        champion_preds = champion_model.predict(X_test)
        challenger_preds = challenger_model.predict(X_test)
        
        champion_rmse = mean_squared_error(y_test, champion_preds, squared=False)
        challenger_rmse = mean_squared_error(y_test, challenger_preds, squared=False)
        
        print("Running model evaluation (champion vs. challenger)")
        print("Challenger: " + str(challenger_rmse))
        print("Champion: " + str(champion_rmse))
        
        # By default register each new model (DEMO ONLY)
#         if challenger_rmse <= champion_rmse:
#             return True
#         else:
#             return False
        
    except Exception as e:
        print(e)
        return True
    
    return True

def register_challenger_model(run_id, model_name):
    """
    Registers a new version of a machine learning model to the MLflow registry. 

    This function registers a new version of a model to the MLflow registry by providing the run_id and model_name. It prints the registered model's version and returns the result object. 

    Args:
    run_id (str): The unique identifier of the MLflow run containing the model.
    model_name (str): The name of the model to be registered. 

    Returns:
    result (mlflow.entities.model_registry.RegisteredModel): An object containing information about the registered model and its version. 

    Example:
    >>> run_id = "1234567890abcdef"
    >>> model_name = "my_model"
    >>> register_challenger_model(run_id, model_name)
    Registering new version of my_model. Version: 1
    <mlflow.entities.model_registry.RegisteredModel object at 0x7f8c9d8e1310>
    """
    result = mlflow.register_model(f"runs:/{run_id}/model", model_name)
    print(f"Registering new version of {model_name}. Version: {str(result.version)}")
    return result

In [ ]:
scoring_script = """
import logging
import os
import json
import mlflow
from io import StringIO
from mlflow.pyfunc.scoring_server import infer_and_parse_json_input, predictions_to_json
from azure.storage.blob import BlobServiceClient, ContainerClient
from datetime import datetime
import pandas as pd
import json
import io
import uuid


def init():
    global model
    global input_schema
    # "model" is the path of the mlflow artifacts when the model was registered. For automl
    # models, this is generally "mlflow-model".
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "model")
    model = mlflow.pyfunc.load_model(model_path)
    input_schema = model.metadata.get_input_schema()


def run(raw_data):
    json_data = json.loads(raw_data)
    if "input_data" not in json_data.keys():
        raise Exception("Request must contain a top level key named 'input_data'")

    serving_input = json.dumps(json_data["input_data"])
    data = infer_and_parse_json_input(serving_input, input_schema)
    predictions = model.predict(data)

    # Logic for home-built model data collector
    conn_str = os.getenv('conn_str')
    container = os.getenv('container')
    now = datetime.now()
    df = pd.DataFrame(json_data['input_data']['data'], columns=json_data['input_data']['columns'])
    df['timestamp'] = datetime.now()

    container_client = ContainerClient.from_connection_string(conn_str, container)
    blob_name = str(uuid.uuid4()) + ".csv"
    output = io.StringIO()
    df.to_csv(output)
    container_client.upload_blob(blob_name, output.getvalue(), overwrite=True)

    result = StringIO()
    predictions_to_json(predictions, result)
    return result.getvalue()
"""

conda_yaml = """
channels:
- conda-forge
dependencies:
- python=3.9.5
- pip<=21.2.4
- pip:
  - mlflow==1.30.0
  - cloudpickle==2.2.1
  - psutil==5.8.0
  - scikit-learn==0.24.2
  - typing-extensions==4.5.0
  - azureml-inference-server-http
  - azure-storage-blob==12.16.0
  - pandas==1.3.4
name: mlflow-env
"""

with open('./score.py', 'w') as file:
    file.write(scoring_script)
    
with open('./conda.yaml', 'w') as file:
    file.write(conda_yaml)

#### Train, evaluate, and register new model

In [ ]:
# Populate Delta table with sample data - Diabetes Dataset
add_data_to_delta_table()

# Load data from Delta table
data = get_data_from_delta_table()

# Establish connection to target Azure ML workspace
ml_client = get_aml_client(subscription_id, resource_group, workspace)

# Get the mlflow tracking URI associated with the AML workspace
mlflow_tracking_uri = get_aml_mlflow_tracking_uri(ml_client)

# Split loaded data into train/test subsets
train_df, test_df = split_train_test_data(data, 0.85)

# Train a new regression model and log to the target workspace. Experiment name/run name are used for organizational purposes and the run ID is returned for subsequent testing/registration
run_id = train_regression_model('DEV-diabetes-model', 'sklearn-gbr', train_df, test_df, 'target', mlflow_tracking_uri)

# Perform champion vs. challenger test to evaluate whether newly trained model is current best performer
better_performer = evaluate_challenger_model(run_id, model_name, test_df, 'target')

# If so, then add to the model registry
if better_performer:
    result = register_challenger_model(run_id, model_name)

#### Helper functions for model deployment

In [ ]:
def check_if_endpoint_exists(endpoint_name, ml_client):
    """
    Checks if an online endpoint exists in the ML model deployment environment. 

    This function checks if an online endpoint with the specified name exists in the ML deployment environment using the provided ML client. It returns True if the endpoint exists and False otherwise. 

    Args:
    endpoint_name (str): The name of the online endpoint to check for existence.
    ml_client (object): An instance of the ML client used to interact with the ML deployment environment. 

    Returns:
    bool: True if the endpoint exists, False otherwise. 
    """
    try:
        endpoint = ml_client.online_endpoints.get(endpoint_name)
        if endpoint:
            print(f'Endpoint {endpoint_name} exists.')
            return True
        else:
            print(f'Endpoint {endpoint_name} does not exist.')
            return False
    except Exception as e:
        return False
    
def create_endpoint(endpoint_name, endpoint_description, ml_client):
    """
    Creates a new online endpoint for a machine learning model using the specified ML client. 

    This function creates a new online endpoint with the provided name and description using the provided ML client. The endpoint is configured with key-based authentication and system-assigned identity. The function prints a message indicating the creation of the endpoint. 

    Args:
    endpoint_name (str): The name of the online endpoint to be created.
    endpoint_description (str): A description for the online endpoint.
    ml_client (object): An instance of the ML client used to interact with the ML deployment environment. 

    Returns:
    None 
    """
    deployment_client = get_deploy_client(mlflow.get_tracking_uri())  
    endpoint_config = {
        "auth_mode": "key",
        "identity": {
            "type": "system_assigned"
        }
    }
    endpoint_config_path = "endpoint_config.json"
    with open(endpoint_config_path, "w") as outfile:
        outfile.write(json.dumps(endpoint_config))
        
    print(f'Creating endpoint {endpoint_name}')
    endpoint = deployment_client.create_endpoint(
        name=endpoint_name,
        config={"endpoint-config-file": endpoint_config_path},
    )
    return

def get_deployments(endpoint_name, ml_client):
    """
    Retrieves the traffic allocation for a specific online endpoint using the provided ML client. 

    This function fetches the traffic allocation for a given online endpoint using the specified ML client. It returns the traffic allocation as a dictionary where each key-value pair represents the model version and its corresponding percentage of traffic. 

    Args:
    endpoint_name (str): The name of the online endpoint for which to retrieve the traffic allocation.
    ml_client (object): An instance of the ML client used to interact with the ML deployment environment. 

    Returns:
    dict: A dictionary containing the traffic allocation for the online endpoint, with model versions as keys and traffic percentages as values. 
    """
    endpoint = ml_client.online_endpoints.get(endpoint_name)
    return endpoint.traffic

def get_staged_deployment(endpoint_name, ml_client):
    """  
    Get the staged deployment of a given endpoint.  

    This function retrieves the staged deployment with the highest traffic percentage  
    for the specified endpoint. If no deployments are found, it returns None.

    Parameters:  
    endpoint_name (str): The name of the endpoint to retrieve the staged deployment from.  
    ml_client (object): The Machine Learning client instance to interact with the API.  

    Returns:  
    str or None: The name of the staged deployment with the highest traffic percentage,  
                 or None if no deployments are found.  
    """  
    endpoint = ml_client.online_endpoints.get(endpoint_name)
    mirror_traffic = endpoint.mirror_traffic
    if len(mirror_traffic.keys())==0:
        return None
    else:
        staged_deployment = max(mirror_traffic, key=lambda k: mirror_traffic[k])
        return staged_deployment

def update_traffic(deployment_name, endpoint_name, traffic_percent):
    """
    Updates the traffic allocation for a specific deployment within an online endpoint. 

    This function updates the traffic percentage for a given deployment within an online endpoint using the provided deployment_name, endpoint_name, and traffic_percent. The traffic allocation is updated using the deployment_client, and the function returns None. 

    Args:
    deployment_name (str): The name of the deployment for which to update the traffic allocation.
    endpoint_name (str): The name of the online endpoint containing the deployment.
    traffic_percent (int): The new traffic percentage to allocate to the specified deployment. 

    Returns:
    None 
    """
    deployment_client = get_deploy_client(mlflow.get_tracking_uri()) 
    traffic_config = {"traffic": {deployment_name: traffic_percent}}
    traffic_config_path = "traffic_config.json"
    
    with open(traffic_config_path, "w") as outfile:
        outfile.write(json.dumps(traffic_config))
        
    print(f"Updating traffic to {endpoint_name}")
    print(json.dumps(traffic_config))
    deployment_client.update_endpoint(
        endpoint=endpoint_name,
        config={"endpoint-config-file": traffic_config_path},
    )
    return

def update_mirror_traffic(deployment_name, endpoint_name, ml_client, traffic_percent):
    """
    Update the mirror traffic percentage of a deployment in an online endpoint. 

    This function retrieves the online endpoint using the given endpoint_name, and updates the mirror traffic percentage
    of the specified deployment_name within that endpoint. The new traffic percentage is set using the provided
    traffic_percent parameter. 

    Args:
    deployment_name (str): The name of the deployment for which the mirror traffic percentage needs to be updated.
    endpoint_name (str): The name of the online endpoint containing the specified deployment.
    ml_client (MLClient): An instance of MLClient used to manage and interact with the machine learning service.
    traffic_percent (float): The new mirror traffic percentage to be assigned to the deployment. 

    Returns:
    None
    """
    print(f"Updating mirror traffic at {endpoint_name}")
    print(json.dumps({deployment_name: traffic_percent}))
    endpoint = ml_client.online_endpoints.get(endpoint_name)
    endpoint.mirror_traffic = {deployment_name: traffic_percent}
    result = ml_client.begin_create_or_update(endpoint).result()
    return result
    
def create_initial_deployment(deployment_name, endpoint_name, model_name, model_version, ml_client, instance_type="Standard_F4s_v2", instance_count=1):
    """
    Create an initial deployment of a model to an online endpoint. 

    This function creates a deployment of a specified model and model version to an online endpoint using the provided
    deployment_name and endpoint_name. The deployment is configured with the given instance_type and instance_count.
    After the deployment is created, the traffic percentage is updated to 100%. 

    Args:
    deployment_name (str): The name of the deployment to be created.
    endpoint_name (str): The name of the online endpoint where the deployment will be created.
    model_name (str): The name of the model to be deployed.
    model_version (str): The version of the model to be deployed.
    ml_client (MLClient): An instance of MLClient used to manage and interact with the machine learning service.
    instance_type (str, optional): The instance type for the deployment. Defaults to "Standard_F4s_v2".
    instance_count (int, optional): The number of instances for the deployment. Defaults to 1. 

    Returns:
    bool: True if the deployment is created successfully, False otherwise.
    """
    print(f"Creating initial deployment. Endpoint: {endpoint_name}. Deployment: {deployment_name}.")
    deploy_config = {
        "instance_type": instance_type,
        "instance_count": instance_count
    }
    
    deployment_config_path = "deployment_config.json"
    
    with open(deployment_config_path, "w") as outfile:
        outfile.write(json.dumps(deploy_config))
        
    deployment_client = get_deploy_client(mlflow.get_tracking_uri())
    
    tags = {'ModelName': model_name, 'ModelVersion': model_version}
    
    model = ml_client.models.get(name = model_name, version=model_version)
    
    environment = Environment(
        conda_file="./conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    )

    env_vars = {'conn_str': dbutils.secrets.get(scope="amlsecretscope",key="storageconnstr"), 'container': dbutils.secrets.get(scope="amlsecretscope",key="container")}

    scale_settings = TargetUtilizationScaleSettings(
        min_instances=1,
        max_instances=6,
        polling_interval=20,
        target_utilization_percentage = 65
    )

    deployment = ManagedOnlineDeployment(
        name=deployment_name,
        endpoint_name=endpoint_name,
        model=model,
        environment=environment,
        environment_variables=env_vars,
        # scale_settings=scale_settings,
        code_configuration=CodeConfiguration(
            code=".",
            scoring_script="score.py"
        ),
        instance_type=instance_type,
        instance_count=instance_count,
    )
    
    poller = ml_client.online_deployments.begin_create_or_update(deployment)
    
    import time
    
    while poller.done() == False:
        time.sleep(5)
    
    # Await completion here
    
#     deployment = deployment_client.create_deployment(
#         name=deployment_name,
#         endpoint=endpoint_name,
#         model_uri=f"models:/{model_name}/{model_version}",
#         config={"deploy-config-file": deployment_config_path},
#     )
    
    update_traffic(deployment_name, endpoint_name, 100)
    
    return True

def create_new_deployment(deployment_name, endpoint_name, model_name, model_version, ml_client, instance_type="Standard_F4s_v2", instance_count=1):
    """
    Create a new deployment of a model to an online endpoint and update the mirror traffic percentage. 

    This function creates a deployment of a specified model and model version to an online endpoint using the provided
    deployment_name and endpoint_name. The deployment is configured with the given instance_type and instance_count.

    Args:
    deployment_name (str): The name of the deployment to be created.
    endpoint_name (str): The name of the online endpoint where the deployment will be created.
    model_name (str): The name of the model to be deployed.
    model_version (str): The version of the model to be deployed.
    ml_client (MLClient): An instance of MLClient used to manage and interact with the machine learning service.
    instance_type (str, optional): The instance type for the deployment. Defaults to "Standard_F4s_v2".
    instance_count (int, optional): The number of instances for the deployment. Defaults to 1.

    Returns:
    None
    """
    print(f"Creating new deployment. Endpoint: {endpoint_name}. Deployment: {deployment_name}.")
    deploy_config = {
        "instance_type": instance_type,
        "instance_count": instance_count
    }
        
    deployment_config_path = "deployment_config.json"
    
    with open(deployment_config_path, "w") as outfile:
        outfile.write(json.dumps(deploy_config))
        
#     deployment_client = get_deploy_client(mlflow.get_tracking_uri()) 
    
#     deployment = deployment_client.create_deployment(
#         name=deployment_name,
#         endpoint=endpoint_name,
#         model_uri=f"models:/{model_name}/{model_version}",
#         config={"deploy-config-file": deployment_config_path},
#     )

    model = ml_client.models.get(name = model_name, version=model_version)
    
    environment = Environment(
        conda_file="./conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:latest",
    )

    env_vars = {'conn_str': dbutils.secrets.get(scope="amlsecretscope",key="storageconnstr"), 'container': dbutils.secrets.get(scope="amlsecretscope",key="container")}

    scale_settings = TargetUtilizationScaleSettings(
        min_instances=1,
        max_instances=6,
        polling_interval=20,
        target_utilization_percentage = 65
    )

    deployment = ManagedOnlineDeployment(
        name=deployment_name,
        endpoint_name=endpoint_name,
        model=model,
        environment=environment,
        environment_variables=env_vars,
        # scale_settings=scale_settings,
        code_configuration=CodeConfiguration(
            code=".",
            scoring_script="score.py"
        ),
        instance_type=instance_type,
        instance_count=instance_count,
    )
    
    poller = ml_client.online_deployments.begin_create_or_update(deployment)
    
    while poller.done() == False:
        time.sleep(5)
    
    # Await completion here

    return
    
def test_new_deployment(deployment_name, endpoint_name, test_df, target):
    """
    Test a new deployment on an online endpoint using a sample data frame. 

    This function tests the specified deployment_name on an online endpoint using the provided test_df data frame.
    It generates a sample request using the test_df (excluding the target column) and sends it to the deployment.
    The function returns True if the number of scored data points matches the length of the test_df, otherwise, it
    returns False. 

    Args:
    deployment_name (str): The name of the deployment to be tested.
    endpoint_name (str): The name of the online endpoint where the deployment is located.
    test_df (pd.DataFrame): A pandas data frame containing the data to be used for testing the deployment.
    target (str): The name of the target column in the test_df that should be excluded from the input data. 

    Returns:
    bool: True if the test is successful and the number of scored data points matches the length of the test_df,
    False otherwise.
    """
    sample_request = {"input_data": test_df.drop(columns=[target]).to_dict(orient='split')}
    sample_request_path = "sample_data.json"
    
    with open(sample_request_path, "w") as outfile:
        outfile.write(json.dumps(sample_request))
        
    print(f"Testing deployment '{deployment_name}' with {str(len(test_df))} datapoints.")
        
    result = ml_client.online_endpoints.invoke(
        endpoint_name=endpoint_name,
        deployment_name=deployment_name,
        request_file=sample_request_path,
    )
    try:
        scored_data = json.loads(result)
        print(f"{str(len(scored_data))} predictions returned.")
        print(result)
        print(scored_data)
        return True
        if len(scored_data)==len(test_df):
            return True
        else:
            return False
    except Exception as e:
        return False
    
    return False

def get_current_deployment_name(deployments, model_name):
    """
    Get the current active deployment name for a given model from a dictionary of deployments. 

    This function takes a dictionary of deployments and the model_name as input, and returns the name of the current
    active deployment for the specified model. If there are no deployments, the function returns None. If there are
    deployments, the function returns the name of the deployment with the highest traffic percentage. 

    Args:
    deployments (dict): A dictionary containing deployment names as keys and their corresponding traffic percentages as values.
    model_name (str): The name of the model for which the current active deployment name is required. 

    Returns:
    str: The name of the current active deployment for the specified model, or None if there are no deployments.
    """
    if len(deployments.keys())==0:
        return None
    else:
        active_deployment = max(deployments, key=lambda k: deployments[k])
    return active_deployment
 
def get_new_deployment_name(deployments, model_name):
    """
    Get a new deployment name for a given model from a dictionary of deployments. 

    This function takes a dictionary of deployments and the model_name as input, and generates a new deployment name
    for the specified model. If there are no deployments, the function creates a deployment name with a 'BLUE' prefix.
    If there are existing deployments, it checks the prefix of the current active deployment and creates a new deployment
    name with the opposite color prefix (either 'BLUE' or 'GREEN'). 

    Args:
    deployments (dict): A dictionary containing deployment names as keys and their corresponding traffic percentages as values.
    model_name (str): The name of the model for which the new deployment name is required. 

    Returns:
    str: A new deployment name for the specified model with either a 'BLUE' or 'GREEN' prefix, depending on the current active deployment.
    """
    if len(deployments.keys())==0:
        deployment_name = f'BLUE-{model_name}'
    else:
        active_deployment = max(deployments, key=lambda k: deployments[k])
        if 'blue' in active_deployment.lower():
            deployment_name = f'GREEN-{model_name}'
        else:
            deployment_name = f'BLUE-{model_name}'
    return deployment_name.lower()

def remove_current_deployment(deployment_name, endpoint_name, ml_client):
    """
    Remove the current deployment from an online endpoint. 

    This function removes the specified deployment_name from the online endpoint using the provided ml_client. The
    function begins the delete operation but does not wait for it to complete. 

    Args:
    deployment_name (str): The name of the deployment to be removed.
    endpoint_name (str): The name of the online endpoint where the deployment is located.
    ml_client (MLClient): An instance of MLClient used to manage and interact with the machine learning service. 

    Returns:
    None
    """
    print(f"Removing deployment {deployment_name} from endpoint {endpoint_name}")
    ml_client.online_deployments.begin_delete(deployment_name, endpoint_name)
    return
    
        

#### Deploy newly trained model using "safe rollout" pattern

[Details about 'safe rollout' pattern](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-safely-rollout-online-endpoints?view=azureml-api-2&tabs=python).

In [ ]:
# If new model performs better than existing champion (or if no champion exists...)
if better_performer:
    
    # Check if endpoint exists
    endpoint_exists = check_if_endpoint_exists(endpoint_name, ml_client)
    
    # If endpoint does not exist create it
    if not endpoint_exists:
        create_endpoint(endpoint_name, endpoint_description, ml_client)
        
    # Get all deployments to current endpoint
    deployments = get_deployments(endpoint_name, ml_client)
    
    # If no deployments exist on current endpoint, create a deployment and route 100% of traffic to it
    if len(deployments.keys())==0:
        
        # Get deployment name (BLUE + model_name)
        deployment_name = get_new_deployment_name(deployments, model_name)
        
        # Create deployment with 0% traffic allocation
        create_initial_deployment(deployment_name, endpoint_name, model_name, result.version, ml_client)
        
        # Test model endpoint with holdout data (smoke test)
        test_result = test_new_deployment(deployment_name, endpoint_name, test_df, 'target')
        
        # If model passes smoke test, route 100% of traffic to the endpoint
        if test_result:
            update_traffic(deployment_name, endpoint_name, 100)

    # Alternatively, a model has already been deployed to this endpoint and we need to update
    else:
        # Get name of current deployment
        active_deployment_name = get_current_deployment_name(deployments, model_name)
        
        # Get name of new deployment (will be associated with open slot [blue or green])
        deployment_name = get_new_deployment_name(deployments, model_name)
        
        # Create deployment with 0% traffic allocation
        create_new_deployment(deployment_name, endpoint_name, model_name, result.version, ml_client)
        
        # Test model endpoint with holdout data (smoke test)
        test_result = test_new_deployment(deployment_name, endpoint_name, test_df, 'target')
        
        # If model passes smoke test, mirror 10% of all traffic to endpoint
        if test_result:
            update_mirror_traffic(deployment_name, endpoint_name, ml_client, 10)
            
# Manual update to route all traffic to new endpoint as part of a separate process...

### Test your endpoint

In the Azure ML Studio UI you can submit data to your endpoint for manual testing. Try using the data below:
```
{
  "input_data": {"index": [0,1,2],
  "columns": ["age", "sex", "bmi", "bp", "s1", "s2", "s3", "s4", "s5", "s6"],
  "data": [[0.00538306037424807,
    -0.044641636506989,
    -0.0482406250171634,
    -0.0125563519424068,
    0.00118294589619092,
    -0.00663740127664067,
    0.0633666506664982,
    -0.0394933828740919,
    -0.0514005352605825,
    -0.0590671943081523],
   [0.0126481372762872,
    0.0506801187398187,
    0.000260918307477141,
    -0.0114087283893043,
    0.0397096259258226,
    0.0572448849284239,
    -0.0397192078479398,
    0.0560805201945126,
    0.024052583226893,
    0.0320591578182113],
   [0.0380759064334241,
    0.0506801187398187,
    0.00888341489852436,
    0.0425295791573734,
    -0.0428475455662452,
    -0.0210422305189592,
    -0.0397192078479398,
    -0.00259226199818282,
    -0.0181182673078967,
    0.00720651632920303]]}
}
```